# Partie 1

Nous commençons par voir comment PyG stocke les graphes en tenseurs. Ensuite, nous verrons un petit exemple d’utilisation du package OGB.

## 1. PyG Dataset et Data
PyG a deux classes pour stocker et transformer les graphes en Tenseurs. La première est `torch_geometric.datasets` qui contient une variété d’ensemble de données graphes. La deuxième classe `torch_geometric.data` qui permet de manipuler des graphes en tant que tenseurs.


### - PyG Dataset

Exécuter la cellule ci-dessous pour télécharger l'ensemble de données de classification de graphes ENZYMES. Ce dataset contient 600 graphes.

In [ ]:
!pip install torch
!pip install torch-cluster
!pip install torch-geometric
!pip install torch-scatter
!pip install torch-sparse
!pip install torch-spline-conv
!pip install torchaudio
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-cluster: filename=torch_cluster-1.6.3-cp310-cp310-linux_x86_64.whl size=690648 sha256=dfcbfb38431b09617f4c4dbd96d4475c8565e3223e5451071347c9daa380c8b9
  Stored in directory: /root/.cache/pip/wheels/51/78/c3/536637b3cdcc3313aa5e8851a6c72b97f6a01877e68c7595e3
Successfully built torch-cluster
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=495091 sha256=56dddf03a51f08bbe76306c133b2f6cd366e2128351038555681cd3833e687af
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from torch_geometric.datasets import TUDataset

root = './enzymes'
name = 'ENZYMES'

# Téléchargement l'ensemble de données ENZYMES
pyg_dataset= TUDataset('./enzymes', 'ENZYMES')

# Cet Ensemble de données contient 600 graphes
print(pyg_dataset)

ENZYMES(600)


###  Quel est le nombre de classes et le nombre de features dans le dataset ENZYMES?


In [ ]:
##############################Votre Code ici##############################

num_classes = pyg_dataset.num_classes
num_features = pyg_dataset.num_features
##########################################################################
print("{} dataset a {} classes".format(name, num_classes))
print("{} dataset a {} features".format(name, num_features))

ENZYMES dataset a 6 classes
ENZYMES dataset a 3 features


### - PyG Data

In [ ]:
#  pyg_dataset est un dataset de classification de graphes
graph_0 = pyg_dataset[0]
print(graph_0)

Data(edge_index=[2, 168], x=[37, 3], y=[1])


### Quelle est la classe (label) du graphe (index 50 dans l'ensemble de données ENZYMES) ?

In [ ]:
############# Votre code ici ############
idx = 50
label =  0

label = pyg_dataset[idx].y.item()
#########################################
print('Le graphe {} a le label {}'.format(idx, label))

Le graphe 50 a le label 5


### Quelle est le nombre d'arêtes du graphe d'index 100?

In [ ]:
 ############# Votre code ici ############
  ## Remarque:
  ## 1. Vous ne pouvez pas retourner data.num_edges directement
  ## 2. Le graphe est non orienté
idx = 100
num_edges = pyg_dataset[idx].num_edges // 2  # Since the graph is undirected
 #########################################
print('Le graphe {} a {} arêtes'.format(idx, num_edges))

Le graphe 100 a 88 arêtes


## 2. Open Graph Benchmark (OGB)

L’OGB est une collection des ensembles de données réels  de grande échelle utilisés pour l’apprentissage automatique sur les graphes. Ces ensembles de données peuvent être téléchargés, traités, est divisés (entrainement, validation et test) automatiquement en utilisant l’OGB Data Loader. L’OGB permet également d’évaluer les performances des différents modèles d’apprentissage d’une manière unifiée.  L’OGB prend également en charge les ensembles de données et les données PyG.

Nous allons jeter un coup d’œil sur l’ensemble de données `ogbn-arxiv`.

- Exécuter la cellule ci-dessous pour charger l'ensemble de données et le transformer en tenseur.

In [ ]:
import torch_geometric.transforms as T
#!pip install ogb
from ogb.nodeproppred import PygNodePropPredDataset

dataset_name = 'ogbn-arxiv'
# charger le dataset et le transformer en sparse tensor
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
print('Le {} dataset a {} graphe'.format(dataset_name, len(dataset)))

# Extract the graph
data = dataset[0]
print(data)

Le ogbn-arxiv dataset a 1 graphe
Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])


### Quel est le nombre de features dans le graphe ogbn-arxiv ?

In [ ]:
################ Votre code ici ##########

num_features = data.num_features
############################################
print('le graphe a {} features'.format(num_features))

le graphe a 128 features


# Partie 2 : GNN pour la classification de noeuds

Dans cette partie, nous allons construire notre premier réseau de neurones de graphes (GNN) en utilisant PyG et nous allons l'appliquer à la tache de prédiction de propriétés de nœuds (classification de nœuds). Notre modèle sera basé sur l’opérateur de convolution de graphes GCN (Kipf et al.2017). On ne vous demande pas d’implémenter cet opérateur. Vous devez utiliser directement la couche `GCNConv` de PyG.

(Kipf et al.2017) : https://arxiv.org/pdf/1609.02907.pdf


## Setup

In [27]:
import torch
import torch.nn.functional as F
print(torch.__version__)

# la couche GCNConv de PyG
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

2.1.0+cu121


## Chargement et prétraitement de données

In [28]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
data = dataset[0]

# Transformer la matrice d'adjacence en matrice Symétrique
data.adj_t = data.adj_t.to_symmetric()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Si vous utilisez un GPU, le device devrait etre cuda
print('Device: {}'.format(device))

# Diviser les données
data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

Device: cpu


## Le modèle GCN

Nous allons maintenant implémenter le modèle suivant.

![image.png](attachment:image.png)

In [29]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name, transform=T.ToSparseTensor())
data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))
data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, return_embeds=False):
        super(GCN, self).__init__()
        self.convs = torch.nn.ModuleList([GCNConv(input_dim, hidden_dim)])
        self.convs.extend([GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers - 1)])
        self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim) for _ in range(num_layers - 1)])
        self.softmax = torch.nn.LogSoftmax(dim=1)
        self.dropout = dropout
        self.return_embeds = return_embeds
        self.num_layers = num_layers
    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        out = x
        for i in range(self.num_layers):
            out = self.convs[i](out, adj_t)
            if i < self.num_layers - 1:
                out = self.bns[i](out)
                out = F.relu(out)
                out = F.dropout(out, p=self.dropout, training=self.training)
        if not self.return_embeds:
            out = self.softmax(out)

        return out

Device: cpu


In [30]:

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList([GCNConv(input_dim, hidden_dim)])
        self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim)])
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
            self.bns.append(torch.nn.BatchNorm1d(hidden_dim))
        self.convs.append(GCNConv(hidden_dim, output_dim))
        self.bns.append(torch.nn.BatchNorm1d(output_dim))

        self.softmax = torch.nn.LogSoftmax(dim=1) if not return_embeds else None
        self.dropout = dropout
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for conv, bn in zip(self.convs[:-1], self.bns[:-1]):
            x = conv(x, adj_t)
            x = bn(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        x = self.bns[-1](x)
        if not self.return_embeds:
            x = self.softmax(x)
        return x


In [31]:
def train(model, data, train_idx, optimizer, loss_fn):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    loss = loss_fn(out[train_idx], data.y[train_idx].squeeze())
    loss.backward()
    optimizer.step()
    return loss.item()


In [32]:

import numpy as np

@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()
    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1)
    y_true = data.y.view(-1, 1).numpy()
    y_pred = y_pred.view(-1, 1).numpy()
    train_acc = evaluator.eval({
        'y_true': y_true[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': y_true[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': y_true[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc


In [33]:
# Ne changez pas les valeurs des ces arguments
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
args

{'device': 'cpu',
 'num_layers': 3,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.01,
 'epochs': 100}

In [34]:
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [35]:
import copy

# reset the parameters to initial random value
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, optimizer, loss_fn)
  result = test(model, data, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 4.4127, Train: 23.11%, Valid: 30.92% Test: 34.64%
Epoch: 02, Loss: 2.8152, Train: 33.40%, Valid: 39.69% Test: 42.42%
Epoch: 03, Loss: 2.4704, Train: 42.84%, Valid: 44.56% Test: 46.68%
Epoch: 04, Loss: 2.2880, Train: 31.26%, Valid: 25.96% Test: 24.70%
Epoch: 05, Loss: 2.1825, Train: 22.27%, Valid: 11.08% Test: 9.09%
Epoch: 06, Loss: 2.1107, Train: 22.79%, Valid: 11.23% Test: 9.40%
Epoch: 07, Loss: 2.0531, Train: 25.71%, Valid: 16.36% Test: 17.15%
Epoch: 08, Loss: 1.9994, Train: 27.55%, Valid: 20.83% Test: 23.18%
Epoch: 09, Loss: 1.9475, Train: 30.31%, Valid: 24.09% Test: 26.97%
Epoch: 10, Loss: 1.9104, Train: 34.04%, Valid: 27.87% Test: 30.96%
Epoch: 11, Loss: 1.8712, Train: 39.22%, Valid: 34.00% Test: 36.96%
Epoch: 12, Loss: 1.8343, Train: 43.04%, Valid: 37.88% Test: 40.30%
Epoch: 13, Loss: 1.8012, Train: 45.94%, Valid: 40.63% Test: 43.24%
Epoch: 14, Loss: 1.7715, Train: 48.10%, Valid: 42.91% Test: 45.89%
Epoch: 15, Loss: 1.7472, Train: 49.98%, Valid: 45.61% Test: 48.9

In [36]:
best_result = test(best_model, data, split_idx, evaluator)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Best model: Train: 73.14%, Valid: 71.80% Test: 70.21%


# Partie 3 : GNN pour la classification de graphes

Dans cette partie, nous allons créer un réseau de neurones de graphes GNN pour la prédiction des propriétés de graphes (classification de graphes)

## Chargement et prétraitement des données

In [37]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from tqdm.notebook import tqdm

# Chargement du dataset
dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

split_idx = dataset.get_idx_split()

# Afficher le type de la tâche (ça devrait afficher  classification binaire)
print('Task type: {}'.format(dataset.task_type))

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:01<00:00,  2.99it/s]
Processing...


Extracting dataset/hiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 41127/41127 [00:00<00:00, 85017.62it/s]


Converting graphs into PyG objects...


100%|██████████| 41127/41127 [00:02<00:00, 16263.35it/s]


Saving...
Device: cpu
Task type: binary classification


Done!


In [38]:
# chargement des données
# On va entrainer le modèle de classification de graphes sur des batchs de 32 graphes
# Mélanger (Shuffle) l'ordre des graphes dans l'ensemble d'entrainement
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False, num_workers=0)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [39]:
# Ne changez pas ces valeurs
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 30,
}
args

{'device': 'cpu',
 'num_layers': 5,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.001,
 'epochs': 30}

## Le modèle de  classification de graphes

Nous allons maintenant impélementer notre GCN pour la prédiction des propriétés de graphes (classification de graphes)

Nous allons réutiliser le modèle précèdent pour générer les plongements des nœuds et utiliser un Pooling global sur les nœuds pour prédire les propriétés du graphe en entier.

In [40]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool

### GCN to predict graph property
class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout):
        super(GCN_Graph, self).__init__()

        # Load encoders for Atoms in molecule graphs
        self.node_encoder = AtomEncoder(hidden_dim)

        # Node embedding model
        # Note that the input_dim and output_dim are set to hidden_dim
        self.gnn_node = GCN(hidden_dim, hidden_dim,
            hidden_dim, num_layers, dropout, return_embeds=True)

        ############# Your code here ############
        ## Note:
        ## 1. Initialisez  self.pool par global mean pooling layer
        ## Lien vers la documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#global-pooling-layers
        ## (une ligne de code )
        self.pool = global_mean_pool
        #########################################

        # Output layer
        self.linear = torch.nn.Linear(hidden_dim, output_dim)


    def reset_parameters(self):
      self.gnn_node.reset_parameters()
      self.linear.reset_parameters()

    def forward(self, batched_data):
        # Input :  tensor batched_data,
        # Output : tensor for each graph.
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch
        embed = self.node_encoder(x)

        out = None

        ############# Votre code ici ###########################################
        ## Note:
        ## 1. Contruire les plongements de noeuds en utilisant le GCN précédent
        ## 2. Utiliser global pooling layer pour constuire les features du graphe en entier
        ## Lien vers la documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#global-pooling-layers
        ## 3. Utilisez le couche linear layer  pour prédire la propriété du graphe

        out = self.gnn_node(embed, edge_index)
        out = self.pool(out, batch)
        out = self.linear(out)

        ########################################################

        return out


In [41]:
def train(model, device, data_loader, optimizer, loss_fn):

    model.train()
    loss = 0

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
      batch = batch.to(device)

      if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
          pass
      else:
        ## ignorer nan targets (unlabeled) lors du calcul du training loss.
        is_labeled = batch.y == batch.y

        ############# Votre code ici ############
        ## Remarques:
        ## 1. N'oubliez pas mettre les gradients à zero : zero_grad()
        ## 2. Passez data au modèle
        ## 3. Utilisez le masque 'is_labeled' pour filtrer outut et labels
        ## 4. Changez le type du label à float
        ## 5. Passez l'ouput et les étiquettes tranchés à loss_fn

        optimizer.zero_grad()
        pred = model(batch)
        loss = loss_fn(pred[is_labeled], batch.y[is_labeled].float())
        loss.backward()
        optimizer.step()

        #########################################

    return loss.item()


In [42]:
# La fonction d'évaluation
def eval(model, device, loader, evaluator):
    model.eval()
    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict)

In [43]:
model = GCN_Graph(args['hidden_dim'],
            dataset.num_tasks, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

In [44]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  print('Training...')
  loss = train(model, device, train_loader, optimizer, loss_fn)

  print('Evaluating...')
  train_result = eval(model, device, train_loader, evaluator)
  val_result = eval(model, device, valid_loader, evaluator)
  test_result = eval(model, device, test_loader, evaluator)

  train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Training...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

...

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 30, Loss: 0.0238, Train: 83.55%, Valid: 80.51% Test: 75.74%


In [45]:
train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator)[dataset.eval_metric]
test_acc = eval(best_model, device, test_loader, evaluator)[dataset.eval_metric]

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Best model: Train: 83.37%, Valid: 81.23% Test: 75.65%


## Faites des tests avec deux autres couches de global pooling de Pytorch Geometric.

In [ ]:
# Réutilisez le code précédent (en changeant uniquement la couche de pooling)


from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from tqdm.notebook import tqdm

dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
split_idx = dataset.get_idx_split()

print('Task type: {}'.format(dataset.task_type))

train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False, num_workers=0)

args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 30,
}

from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool, global_sort_pool, global_max_pool

class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout):
        super(GCN_Graph, self).__init__()
        self.node_encoder = AtomEncoder(hidden_dim)
        self.gnn_node = GCN(hidden_dim, hidden_dim, hidden_dim, num_layers, dropout, return_embeds=True)
        self.pool = global_sort_pool
        self.linear = torch.nn.Linear(hidden_dim, output_dim)


    def reset_parameters(self):
      self.gnn_node.reset_parameters()
      self.linear.reset_parameters()

    def forward(self, batched_data):
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch
        embed = self.node_encoder(x)
        out = self.gnn_node(embed, edge_index)
        out = self.pool(out, batch)
        out = self.linear(out)

        return out

def train(model, device, data_loader, optimizer, loss_fn):
    model.train()
    loss = 0

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
      batch = batch.to(device)

      if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
          pass
      else:
        is_labeled = batch.y == batch.y

        optimizer.zero_grad()
        pred = model(batch)
        loss = loss_fn(pred[is_labeled], batch.y[is_labeled].float())
        loss.backward()
        optimizer.step()

    return loss.item()

def eval(model, device, loader, evaluator):
    model.eval()
    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict)

model = GCN_Graph(args['hidden_dim'], dataset.num_tasks, args['num_layers'], args['dropout']).to(device)
evaluator = Evaluator(name='ogbg-molhiv')
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  print('Training...')
  loss = train(model, device, train_loader, optimizer, loss_fn)

  print('Evaluating...')
  train_result = eval(model, device, train_loader, evaluator)
  val_result = eval(model, device, valid_loader, evaluator)
  test_result = eval(model, device, test_loader, evaluator)

  train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')
train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator)[dataset.eval_metric]
test_acc = eval(best_model, device, test_loader, evaluator)[dataset.eval_metric]

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')
